In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [20]:
data=pd.read_csv('updated_pollution_dataset.csv')
data

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,Air Quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,Moderate
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,Moderate
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,Moderate
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,Good
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,Good
...,...,...,...,...,...,...,...,...,...,...
4995,40.6,74.1,116.0,126.7,45.5,25.7,2.11,2.8,765,Hazardous
4996,28.1,96.9,6.9,25.0,25.3,10.8,1.54,5.7,709,Moderate
4997,25.9,78.2,14.2,22.1,34.8,7.8,1.63,9.6,379,Moderate
4998,25.3,44.4,21.4,29.0,23.7,5.7,0.89,11.6,241,Good


In [28]:
# Encode target (Air Quality) into numeric values
le = LabelEncoder()
data["Air Quality"] = le.fit_transform(data["Air Quality"])
# Features and Target
X = data.drop(columns=["Air Quality"])
y = data["Air Quality"]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [30]:
# Bat Algorithm Parameters
num_bats = 10
max_iter =20
loudness = 0.6
pulse_rate = 0.5
frequency_min, frequency_max = 0, 2

# Fitness Function with Additional Metrics
def fitness_function_with_metrics(solution):
    selected_features = np.where(solution > 0.5)[0]
    if len(selected_features) == 0:
        return 0, 0, 0, 0  # Return 0 for accuracy, precision, recall, and F1 if no features are selected

    # Train model on selected features
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train.iloc[:, selected_features], y_train)
    preds = clf.predict(X_test.iloc[:, selected_features])

   # Calculate metrics
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds, average='macro')  # Use 'macro' for multiclass
    recall = recall_score(y_test, preds, average='macro')
    f1 = f1_score(y_test, preds, average='macro')

    return accuracy, precision, recall, f1

# Initialize bats
positions = np.random.rand(num_bats, X.shape[1])  # Random positions in [0,1]
velocities = np.random.uniform(-1, 1, (num_bats, X.shape[1]))  # Random velocities
best_global = positions[np.random.randint(0, num_bats)]  # Randomly select a bat
best_global_accuracy, best_global_precision, best_global_recall, best_global_f1 = fitness_function_with_metrics(best_global)




In [32]:
# Bat Algorithm
for t in range(max_iter):
    for i in range(num_bats):
        # Calculate frequency and update velocity and position
        frequency = frequency_min + (frequency_max - frequency_min) * np.random.rand()
        velocities[i] += (positions[i] - best_global) * frequency
        positions[i] = np.clip(positions[i] + velocities[i], 0, 1)  # Ensure positions stay in range

        # Local search
        if np.random.rand() > pulse_rate:
            positions[i] = np.clip(best_global + np.random.normal(0, 0.1, size=X.shape[1]), 0, 1)

        # Evaluate fitness
        accuracy, precision, recall, f1 = fitness_function_with_metrics(positions[i])
        if accuracy > best_global_accuracy and np.random.rand() < loudness:
            best_global = positions[i]
            best_global_accuracy, best_global_precision, best_global_recall, best_global_f1 = accuracy, precision, recall, f1

    # Update loudness and pulse rate
    loudness = max(0.1, loudness * 0.95)
    pulse_rate = min(1.0, pulse_rate * 1.05)

    # Print progress
    print(f"Iteration {t + 1}:")
    print(f"   Best Accuracy = {best_global_accuracy:.4f}")
    print(f"   Best Precision = {best_global_precision:.4f}")
    print(f"   Best Recall = {best_global_recall:.4f}")
    print(f"   Best F1 Score = {best_global_f1:.4f}")
    selected_features = np.where(best_global > 0.5)[0]
    print("   Selected Features:", X.columns[selected_features].tolist())
    print()

# Final Results
print("Final Results:")
print(f"Best Accuracy: {best_global_accuracy:.4f}")
print(f"Best Precision: {best_global_precision:.4f}")
print(f"Best Recall: {best_global_recall:.4f}")
print(f"Best F1 Score: {best_global_f1:.4f}")
print("Selected Features:", X.columns[selected_features].tolist())

Iteration 1:
   Best Accuracy = 0.9273
   Best Precision = 0.8990
   Best Recall = 0.8786
   Best F1 Score = 0.8867
   Selected Features: ['PM2.5', 'PM10', 'CO', 'Proximity_to_Industrial_Areas', 'Population_Density']

Iteration 2:
   Best Accuracy = 0.9367
   Best Precision = 0.9117
   Best Recall = 0.9005
   Best F1 Score = 0.9055
   Selected Features: ['PM2.5', 'PM10', 'NO2', 'CO', 'Proximity_to_Industrial_Areas', 'Population_Density']

Iteration 3:
   Best Accuracy = 0.9420
   Best Precision = 0.9235
   Best Recall = 0.9049
   Best F1 Score = 0.9123
   Selected Features: ['Humidity', 'PM2.5', 'PM10', 'NO2', 'CO', 'Proximity_to_Industrial_Areas', 'Population_Density']

Iteration 4:
   Best Accuracy = 0.9420
   Best Precision = 0.9235
   Best Recall = 0.9049
   Best F1 Score = 0.9123
   Selected Features: ['Humidity', 'PM2.5', 'PM10', 'NO2', 'CO', 'Proximity_to_Industrial_Areas', 'Population_Density']

Iteration 5:
   Best Accuracy = 0.9420
   Best Precision = 0.9235
   Best Recall = 